In [1]:
# Base Imports
import os
import json
import numpy as np
import pandas as pd


# WUBRG Imports
import WUBRG
from WUBRG import COLOR_ALIASES_SUPPORT, COLOR_ALIASES, COLOUR_GROUPINGS, COLOUR_GROUPS, COLORS


# set_info imports
import set_info
from set_info import SETS, FORMATS, STAT_NAMES
DATA_CACHE = dict()
PANDAS_CACHE = dict()
WINRATES = dict()
METAGAME_REPORT = dict()


# Function pointers to moved functions.
min_play_filter = set_info.min_play_filter
color_id_filter = set_info.color_id_filter
rarity_filter = set_info.rarity_filter
color_game_counts = set_info.color_game_counts
get_top = set_info.get_top
gen_metadata_dict = set_info.gen_metadata_dict
get_color_group_metadata = set_info.get_color_group_metadata
get_color_overview = set_info.get_color_overview
get_format_metagame_data = set_info.get_format_metagame_data
get_card_frame = set_info.get_card_frame

print(f'SETS:          {str(SETS)}')
print(f'FORMATS:       {str(FORMATS)}')

COLUMNS_TRUNC = ["Color", "Rarity", "ALSA", "# GP", "GP WR", "# GIH", "GIH WR"]

WRANING: 'discord' failed to import. Emoji support unavailable
SETS:          ['MID']
FORMATS:       ['PremierDraft', 'TradDraft']


In [2]:
s = SETS[0]
f = FORMATS[0]

## Filters

In [3]:
# # Filters out cards that have a number of games played less than p% of # GP.
# def min_play_filter(df, p):
#     min_game = df['# GP'].sum() * (p/100)
#     return df[df['# GP'] >= min_game]

# # Filters out cards that don't fall within the colour identy.
# def color_id_filter(df, color_string):
#     if color_string is not None and color_string != '':
#         return df[df['Color'].apply(lambda x: set(x) <= set(color_string))]
#     else:
#         return df

# # Filters out cards that aren't in the given rarities.
# def rarity_filter(df, rarity):
#     if rarity is not None and rarity != '':
#         return df[df['Rarity'].apply(lambda x: set(x) <= set(rarity))]
#     else:
#         return df

In [5]:
# # Gets the top n cards, based on a particular stat column.
# # Can filter based on card colours, rarity.
# # Can get the bottom n cards with 'reverse=True'
# def get_top(df, stat, n=5, card_colors=None, card_rarity=None, min_thresh=1, reverse=False, columns=None):
#     if df.empty:
#         return df

#     if columns == None:
#         columns = list(df)
    
#     filtered = color_id_filter(df, card_colors)
#     filtered = rarity_filter(filtered, card_rarity)
#     filtered = min_play_filter(filtered, min_thresh)

#     # If reverse is true, we need to invert the ordering.
#     # If we're getting card pick info, we also need to invert the ordering.
#     smallest = reverse != (stat == "ALSA" or stat == "ATA")  # != is a functional XOR.
    
#     # Return the smallest values if we're dealing with pick orders
#     if smallest:
#         filtered = filtered.nsmallest(n, stat)
#     else:
#         filtered = filtered.nlargest(n, stat)


#     if reverse:
#         filtered.columns.name = f"'Top {n} by {stat}'"
#     else:
#         filtered.columns.name = f"'Bottom {n} by {stat}'"

    
#     # TODO: Give a clearer idea of the restrictions on the data in the DataFrame.
#     #c_filter = f"\r\nColours: '{card_colors}'" if card_colors is not None else ""
#     #r_filter = f"\r\nRarities: '{card_rarity}'" if card_colors is not None else ""
    
#     #filtered.columns.name = title + c_filter + r_filter
#     filtered = filtered[columns]
#     return filtered

In [114]:
# # Gets a dataframe for the card, where the deck colours are the indexes.
# def get_card_frame(s, f, card_name):
#     columns = list(PANDAS_CACHE[s][f][''])
#     card_df = pd.DataFrame(columns=columns)
    
#     for c in PANDAS_CACHE[s][f]:
#         df = PANDAS_CACHE[s][f][c]
#         if c == '':
#             c = 'Overall'
#         data_df = None
#         if df.empty or card_name not in df.index:
#             data_df = pd.DataFrame(np.nan, index=[c], columns=columns)
#         else:
#             series = df.loc[card_name]
#             data_df = series.to_frame(name=c).transpose()
#         card_df = card_df.append(data_df)
    
#     return card_df

## Metagame Info

In [4]:
# def color_game_counts(s, f, c, color_filter = None):
#     df = PANDAS_CACHE[s][f][c]
#     if df.empty:
#         return (np.nan, np.nan, np.nan)
    
#     df = color_id_filter(df, color_filter)
#     df = df[['# GP','GP WR']]
#     games_played = df['# GP'].sum()
#     games_won = (df['# GP'] * (df['GP WR'] * 0.01)).sum().round()
#     percent_won = ((games_won  * 100)/ games_played).round(2)
#     return (percent_won, games_won, games_played)

# def get_color_win_rates():
#     win_rates = set_info.get_set_tree()
#     for s in SETS:
#         for f in FORMATS:
#             for c in COLOUR_GROUPS:
#                 win_rates[s][f][c] = color_game_counts(s,f,c)
#     global WINRATES
#     WINRATES = win_rates

In [6]:
# def gen_metadata_dict():
#     ret = {
#         'Winrate' : None,
#         'Winrate Delta' : None,
#         'Games Played' : None, 
#         'Metagame Share' : None,
#         'Top Commons' : None,
#         'Top Uncommons' : None
#     }
#     return ret

# # Get metagame data for a given colour group.
# def get_color_group_metadata(s, f, colors, n=5, stat='GIH WR', columns=None):
#     # Set up the data.
#     c = WUBRG.get_color_identity(colors)
#     metagame_data = gen_metadata_dict()
    
#     # Get the frame, and handle column filtering.
#     df = PANDAS_CACHE[s][f][c]
#     if columns == None:
#         columns = list(df)
    
#     # Get the overall stats, and the stats for the colour group.
#     overall_games = color_game_counts(s, f, '')
#     color_games = color_game_counts(s, f, c)
    
#     # Use those stats to get high-level stats for the colours.
#     metagame_data['Winrate'] = color_games[0]
#     metagame_data['Winrate Delta'] = (color_games[0] - overall_games[0]).round(2)
#     metagame_data['Games Played'] = color_games[2]
#     metagame_data['Metagame Share'] = ((color_games[2] / overall_games[2]) * 100).round(2)
    
#     # Get top commons and uncommons for the colour group.
#     metagame_data['Top Commons'] = get_top(df, stat, n=n, card_rarity='C', columns=columns)
#     metagame_data['Top Uncommons'] = get_top(df, stat, n=n, card_rarity='U', columns=columns)
        
#     return metagame_data

# # Gets a suite of data about the given colour's position in the metagame.
# def get_color_overview(s, f, main_color, stat='GIH WR', columns=None):
#     # Set up the data.
#     metagame_dict = dict()
#     metagame_data = gen_metadata_dict()
#     metagame_dict[main_color] = metagame_data
    
#     # Get the frame, and handle column filtering.
#     df = PANDAS_CACHE[s][f]['']    
#     if columns == None:
#         columns = list(df)
    
#     # Get the overall stats, and the stats filtered on the main_colour.
#     overall_games = color_game_counts(s, f, '')
#     color_games = color_game_counts(s, f, '', main_color)
    
#     # Use those stats to get high-level stats for the colours.
#     metagame_data['Winrate'] = color_games[0]
#     metagame_data['Winrate Delta'] = (color_games[0] - overall_games[0]).round(2)
#     metagame_data['Games Played'] = color_games[2]
#     metagame_data['Metagame Share'] = ((color_games[2] / overall_games[2]) * 100).round(2)
    
#     # Get top commons and uncommons for the main_colour
#     print(f'{s} {f} {main_color} {stat}')
#     metagame_data['Top Commons'] = get_top(df, stat, n=10, card_colors=main_color, card_rarity='C', columns=columns)
#     metagame_data['Top Uncommons'] = get_top(df, stat, n=10, card_colors=main_color, card_rarity='U', columns=columns)
    
#     # Get the metagame data for all colour pairs that contain 'color'.
#     color_pairs = [main_color + x for x in COLORS if x != main_color]
#     for col in color_pairs:
#         c = WUBRG.get_color_identity(col)
#         metagame_dict[c] = get_color_group_metadata(s, f, c, stat=stat, columns=columns)
        
#     # TODO: Add in a fun fact here.
#     print(f'{s} {f} {main_color} {stat}')

#     return metagame_dict

In [7]:
# # Gets an overview of the metagame data and saves it to METAGAME_REPORT
# def get_format_metagame_data(s, f, stat='GIH WR', columns=None):
#     if columns is None:
#         columns = COLUMNS_TRUNC
    
#     report = dict()
#     for s in SETS:
#         report[s] = dict()
#         for f in FORMATS:
#             report[s][f] = dict()
#             for c in COLORS:
#                 report[s][f][c] = get_color_overview(s, f, c, stat=stat, columns=columns)
#     return report

In [8]:
set_info.init_cache()
DATA_CACHE = set_info.DATA_CACHE
PANDAS_CACHE = set_info.PANDAS_CACHE

Checking for new format data...
MID PremierDraft data is up to date!
MID TradDraft data is up to date!
Done checking for new format data.

Checking for cache updates...
Parsing MID_PremierDraft.json...
MID PremierDraft added to DATA_CACHE!
Parsing MID_TradDraft.json...
MID TradDraft added to DATA_CACHE!
Done checking for cache updates.



In [9]:
WINRATES = set_info.WINRATES
WINRATES

{'MID': {'PremierDraft': {'': (55.74, 3313478.0, 5944279),
   'W': (57.84, 7710.0, 13331),
   'U': (59.05, 7798.0, 13206),
   'B': (55.81, 3654.0, 6547),
   'R': (53.43, 5458.0, 10216),
   'G': (57.74, 6487.0, 11235),
   'WU': (58.38, 373926.0, 640523),
   'WB': (56.43, 408979.0, 724799),
   'WR': (54.06, 173120.0, 320249),
   'WG': (56.71, 302387.0, 533188),
   'UB': (58.05, 832884.0, 1434654),
   'UR': (52.01, 172425.0, 331527),
   'UG': (56.72, 239685.0, 422569),
   'BR': (54.22, 290214.0, 535221),
   'BG': (54.28, 184842.0, 340546),
   'RG': (51.17, 168174.0, 328645),
   'WUR': (46.48, 4367.0, 9395),
   'WBR': (46.58, 9587.0, 20583),
   'WBG': (49.08, 15602.0, 31787),
   'UBG': (50.33, 31165.0, 61920),
   'URG': (49.53, 10410.0, 21016),
   'WUB': (50.14, 25306.0, 50468),
   'WUG': (50.91, 12407.0, 24369),
   'WRG': (48.0, 9805.0, 20428),
   'UBR': (45.18, 6776.0, 14997),
   'BRG': (46.3, 9370.0, 20239),
   'WUBR': (0.0, 0.0, 46),
   'WUBG': (48.84, 189.0, 387),
   'WURG': (36.9, 48

In [14]:
METAGAME_REPORT = get_format_metagame_data(s, f, columns=COLUMNS_TRUNC)

for c in METAGAME_REPORT[s][f]:
    string = f'{c}: ['
    for g in METAGAME_REPORT[s][f][c]:
        d = METAGAME_REPORT[s][f][c][g]
        wr = "%{:05.2f}".format(d['Winrate'])
        ms = "%{:05.2f}".format(d['Metagame Share'])
        string += f'{g} ({wr}, {ms}), '
    string = string[:-2] + ']'
    print(string)

W: [W (%56.09, %21.06), WU (%58.38, %10.78), WB (%56.43, %12.19), WR (%54.06, %05.39), WG (%56.71, %08.97)]
U: [U (%56.85, %26.36), WU (%58.38, %10.78), UB (%58.05, %24.14), UR (%52.01, %05.58), UG (%56.72, %07.11)]
B: [B (%56.17, %26.87), WB (%56.43, %12.19), UB (%58.05, %24.14), BR (%54.22, %09.00), BG (%54.28, %05.73)]
R: [R (%53.34, %15.54), WR (%54.06, %05.39), UR (%52.01, %05.58), BR (%54.22, %09.00), RG (%51.17, %05.53)]
G: [G (%54.73, %16.15), WG (%56.71, %08.97), UG (%56.72, %07.11), BG (%54.28, %05.73), RG (%51.17, %05.53)]


In [11]:
def format_metadata_dict(meta_dict):
    text = ''

    return text

def format_colour_overview(meta_dict):
    text = ''
    return text
    
def export_metagame_data():
    text = ''
    for c in COLORS:
        text += format_colour_overview(METAGAME_REPORT[c])
    return text

## Card Evaluation Functions

In [ ]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_qudrant_cards_df(df, rarity, iwd_thresh, func1, func2, cols=None):
    by_rarity = df[df['Rarity'] == rarity]
    games_played_mean = by_rarity['# GP'].mean()
    cards = by_rarity[func1(by_rarity['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    if cols is not None:
        cards = cards[cols]
    return cards


def get_trap_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, gt, cols)

def get_trap_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_trap_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_niche_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, lt, cols)

def get_niche_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_niche_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_staple_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, gt, gt, cols)

def get_staple_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_staple_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)


def get_dreg_cards_df(df, rarity='common', iwd_thresh=0, cols=None):
    return filter_qudrant_cards_df(df, rarity, iwd_thresh, lt, lt, cols)

def get_dreg_cards(s, f, c, rarity='common', iwd_thresh=0, cols=None):
    return get_dreg_cards_df(PANDAS_CACHE[s][f][c], rarity, iwd_thresh, cols)

In [ ]:
colours = ''
rarity = 'U'
get_trap_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_niche_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_staple_cards('MID', 'PremierDraft', colours, rarity, 0.005)

In [ ]:
colours = ''
rarity = 'U'
get_dreg_cards('MID', 'PremierDraft', colours, rarity, -0.005)

## Testing

In [170]:
def get_card_main_stats(s, f, card_name):
    card_frame = get_card_frame(s, f, card_name)
    card_colors = card_frame['Color'].loc['Overall']
    indexes = ['Overall']
    if len(card_colors) == 1:
        indexes = indexes + get_color_supersets(card_colors, 2)
    else:
        indexes = indexes + [card_colors]
    return card_frame.loc[indexes]

,Color,Rarity,# Seen,ALSA,# Picked,ATA,# GP,GP WR,# OH,OH WR,# GD,GD WR,# GIH,GIH WR,# GND,GND WR,IWD
Overall,U,C,78362,4.267,31793,4.674,105762,58.853,29612,61.367,55701,63.162,85313,62.539,43897,54.815,7.724
WU,U,C,78362,4.267,31793,4.674,21554,60.272,5832,63.443,11181,63.492,17013,63.475,8941,56.705,6.77
UB,U,C,78362,4.267,31793,4.674,53467,59.893,15337,62.013,29477,64.284,44814,63.507,21894,55.696,7.811
UR,U,C,78362,4.267,31793,4.674,9793,54.427,2648,56.949,4852,60.202,7500,59.053,4060,50.197,8.856
UG,U,C,78362,4.267,31793,4.674,14852,58.1,4268,61.364,7218,61.554,11486,61.484,6329,54.211,7.273


In [144]:
card_frame['Color'].loc['Overall']

'R'

In [ ]:
def center(df, stat):
    temp = df.copy()
    
    stat_mean = temp[stat].mean()
    temp[stat] = temp[stat] - stat_mean
    return temp.sort_values(stat, ascending=(stat == "ALSA" or stat == "ATA"))

In [ ]:
center(PANDAS_CACHE['MID']['PremierDraft']['WU'], 'GIH WR')